In [1]:
import pandas as pd

In [2]:
data_model_output = pd.read_csv('../data/processed/data_model_output.csv')

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45832\1409545975.py:1: DtypeWarning: Columns (1,8,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,40,41,47,52,58,63,67,69,71,73,77,84,86,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,136,137,140,141,149,151,157,159,161,173,177,186,187,190,193,197,212,214,215,216,217,218,219,220,221,222,223,246,253,254,258,259,261,262,280,298,300,301,303,319,321,343) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_output = pd.read_csv('../data/processed/data_model_output.csv')


In [3]:
data_model_output.head()

,unique_id,respondentid,is_completed,date_completed,is_pilot,is_self_administered,record_type_synthetic,access_mode,access_mode_label,access_mode_other,...,trip_start_time_label,valid_record_person,valid_record_trip,validation_error_person,validation_error_trip,validation_num_errors_person,validation_num_errors_trip,validation_severity_person,validation_severity_trip,weight
0,1,5473,True,2024-10-04,False,False,0,NaN,NaN,NaN,...,FIVE_TO_FIVE_THIRTY,True,True,[],[],0,0,NaN,NaN,8.219240
1,2,5476,True,2024-10-04,False,False,0,1.0,WALK,NaN,...,EIGHT_THIRTY_TO_NINE,True,True,[],[],0,0,NaN,NaN,7.038277
2,3,5489,True,2024-10-04,False,False,0,NaN,NaN,NaN,...,SIX_TO_SIX_THIRTY,True,True,[],[],0,0,NaN,NaN,8.219240
3,4,5558,True,2024-10-04,False,False,0,1.0,WALK,NaN,...,ELEVEN_TO_ELEVEN_THIRTY,True,True,[],[],0,0,NaN,NaN,7.038277
4,5,5593,True,2024-10-04,False,False,0,1.0,WALK,NaN,...,NOON_TO_TWELVE_THIRTY,True,True,[],[],0,0,NaN,NaN,7.038277


In [4]:
import pandas as pd

# Filter data for relevant passenger segments
filtered_data = data_model_output[
    data_model_output['passenger_segment_label'].isin(['RESIDENT_ARRIVING', 'RESIDENT_DEPARTING'])
]

# Create new columns for municipal_zone and pmsa based on passenger segment type
filtered_data['municipal_zone'] = filtered_data.apply(
    lambda row: row['destination_municipal_zone'] if row['passenger_segment_label'] == 'RESIDENT_ARRIVING' else row['origin_municipal_zone'],
    axis=1
)

filtered_data['pmsa'] = filtered_data.apply(
    lambda row: row['destination_pmsa'] if row['passenger_segment_label'] == 'RESIDENT_ARRIVING' else row['origin_pmsa'],
    axis=1
)

# Adjust party size and create weighted party size
filtered_data['party_size_flight'] += 1
filtered_data['party_size_weighted'] = filtered_data['party_size_flight'] * filtered_data['weight']

# Define a function to generate the required summaries
def generate_summaries(group_col):
    summary_weight = filtered_data.groupby(group_col).agg(
        weighted_num_travelers = ('party_size_weighted', 'sum'),
        weighted_num_parties = ('weight', 'sum')
    ).reset_index()
    
    summary_respondents = filtered_data[filtered_data['record_type_synthetic'] == 0].groupby(group_col).agg(
        num_travelers = ('party_size_flight', 'sum'),
        num_travel_parties = ('unique_id', 'count')
    ).reset_index()
    
    return summary_weight, summary_respondents

# Generate summaries for municipal_zone
summary_municipal_zone_weight, summary_municipal_zone_respondents = generate_summaries('municipal_zone')

# Generate summaries for pmsa
summary_pmsa_weight, summary_pmsa_respondents = generate_summaries('pmsa')

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45832\3699141501.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['municipal_zone'] = filtered_data.apply(
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45832\3699141501.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['pmsa'] = filtered_data.apply(
C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45832\3699141501.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [5]:
summary_municipal_zone_weight

,municipal_zone,weighted_num_travelers,weighted_num_parties
0,CARLSBAD,526.844200,292.488590
1,CHULA VISTA,2361.191361,711.334718
2,CORONADO,213.096569,93.642618
3,DEL MAR,86.786378,51.846977
4,EL CAJON,420.023077,194.973387
5,ENCINITAS,374.655375,172.193913
6,ESCONDIDO,1177.077348,338.495664
7,EXTERNAL,3668.026533,1038.556967
8,IMPERIAL BEACH,38.011882,28.336691
9,LA MESA,171.648016,116.559005


In [6]:
summary_municipal_zone_respondents

,municipal_zone,num_travelers,num_travel_parties
0,CARLSBAD,102.0,70
1,CHULA VISTA,187.0,102
2,CORONADO,32.0,19
3,DEL MAR,18.0,13
4,EL CAJON,70.0,39
5,ENCINITAS,56.0,40
6,ESCONDIDO,80.0,43
7,EXTERNAL,309.0,156
8,IMPERIAL BEACH,10.0,8
9,LA MESA,40.0,30


In [7]:
summary_pmsa_respondents

,pmsa,num_travelers,num_travel_parties
0,1.0,105.0,62
1,2.0,565.0,350
2,3.0,685.0,444
3,4.0,223.0,130
4,5.0,246.0,148
5,6.0,294.0,209
6,7.0,189.0,101
7,8.0,6.0,4
8,99.0,309.0,156


In [8]:
summary_pmsa_weight

,pmsa,weighted_num_travelers,weighted_num_parties
0,1.0,1427.068213,428.054941
1,2.0,4579.221813,1814.357443
2,3.0,5010.173047,2133.674078
3,4.0,2512.470794,816.775290
4,5.0,1949.735050,767.673576
5,6.0,1756.762086,900.913421
6,7.0,2305.501442,704.702735
7,8.0,57.129161,25.834206
8,99.0,3668.026533,1038.556967


In [9]:
# Merge summaries for municipal_zone
summary_municipal_zone = summary_municipal_zone_weight.merge(
    summary_municipal_zone_respondents, on='municipal_zone', how='outer'
)

# Merge summaries for pmsa
summary_pmsa = summary_pmsa_weight.merge(
    summary_pmsa_respondents, on='pmsa', how='outer'
)

# Write to an Excel file with two sheets
with pd.ExcelWriter("../reports/zonal_distribution_resident_passengers.xlsx") as writer:
    summary_municipal_zone.to_excel(writer, sheet_name="Municipal Zone Summary", index=False)
    summary_pmsa.to_excel(writer, sheet_name="PMSA Summary", index=False)